In [1]:
import os
import cv2
import torch
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
from rembg import remove  # Correct import for rembg

In [7]:
input_dir = "D:\MED_LEAF_ID-1\dataset\Medicinal Leaf dataset"
output_dir = "D:/MED_LEAF_ID/preprocessed_glcm"
os.makedirs(output_dir, exist_ok=True)

In [8]:
allowed_extensions = {".jpg", ".jpeg", ".png", ".bmp"}

In [9]:
def preprocess_image(image):
    """ Applies CLAHE, Gaussian Blur, Background Removal using rembg, and Edge Detection """
    
    # Convert to PIL for rembg
    pil_image = Image.fromarray(image)
    
    # Remove background using rembg
    output_pil = remove(pil_image)
    
    # Convert back to NumPy (RGBA)
    output_np = np.array(output_pil)
    
    # If the image has an alpha channel (RGBA), remove it
    if output_np.shape[2] == 4:  # Check if the image is RGBA
        output_np = cv2.cvtColor(output_np, cv2.COLOR_BGRA2BGR)  # Convert to RGB (3 channels)

    # Convert to Grayscale
    gray = cv2.cvtColor(output_np, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE (Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)

    # Apply Gaussian Blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Edge detection using Sobel operator to highlight edges
    edges = cv2.Sobel(blurred, cv2.CV_64F, 1, 1, ksize=3)
    edges = cv2.convertScaleAbs(edges)

    # Combine edges with the blurred image for enhanced texture features
    combined = cv2.addWeighted(blurred, 0.7, edges, 0.3, 0)

    # Normalize the image
    normalized = cv2.normalize(combined, None, 0, 255, cv2.NORM_MINMAX)

    # Remove Background using Thresholding
    _, mask = cv2.threshold(normalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operation to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Set background to white where mask is 0
    output_np[mask == 0] = (255, 255, 255)  # Set background to white

    return output_np

In [10]:
augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),  # Reduced degree for less distortion
    transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Reduced jitter range
    transforms.GaussianBlur(kernel_size=5),  # Increased kernel size for stronger blur
])

In [11]:
for root, _, files in os.walk(input_dir):
    for filename in files:
        if os.path.splitext(filename)[1].lower() in allowed_extensions:  # ✅ Fixed Error
            img_path = os.path.join(root, filename)

            # Read Image
            image = cv2.imread(img_path)

            if image is None:
                print(f"❌ Skipping unreadable image: {img_path}")
                continue

            # Apply Preprocessing with rembg and CLAHE/Blur/Edge Detection
            processed_img = preprocess_image(image)

            # Convert to PIL for augmentation
            processed_pil = Image.fromarray(processed_img)

            # Apply Augmentation using Torchvision
            augmented_pil = augmentations(processed_pil)

            # Convert back to NumPy
            augmented_np = np.array(augmented_pil)

            # Generate output path preserving folder structure
            relative_path = os.path.relpath(img_path, input_dir)
            save_path = os.path.join(output_dir, relative_path)

            # Ensure subdirectory exists
            os.makedirs(os.path.dirname(save_path), exist_ok=True)

            # Save Processed and Augmented Images
            cv2.imwrite(save_path, augmented_np)

            print(f"✅ Saved: {save_path}")

print("🔥 GLCM Preprocessing & Augmentation (Torchvision) Completed!")

✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\10.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\100.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\102.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\104.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\106.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\108.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\110.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\112.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\114.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\116.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\118.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\12.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\120.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\122.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\124.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\126.jpg
✅ Saved: D:/MED_LEAF_ID/preprocessed_glcm\Aloevera\128.jpg